In [11]:
# LeetCode 113. Path Sum II
# Time Complexity: O(n)
# Space Complexity: O(h)

# 113. Path Sum II

[Link to Problem](https://leetcode.com/problems/path-sum-ii/description/)

### Description
Given the `root` of a binary tree and an integer `targetSum`, return all **root-to-leaf** paths where the sum of the node values in the path equals `targetSum`.

Each path should be returned as a list of the node values, **not** node references.

A **leaf** is a node with no children.

---
**Example 1:**

Input: `root = [5,4,8,11,null,13,4,7,2,null,null,5,1]`, `targetSum = 22`
Output: `[[5,4,11,2],[5,8,4,5]]`

![example1](pathsumii1.jpg)

**Example 2:**

Input: `root = [1,2,3]`, `targetSum = 5`
Output: `[]`

![example2](pathsum2.jpg)


**Example 3:**

Input: `root = [1,2]`, `targetSum = 0`
Output: `[]`

---
**Constraints:**
- The number of nodes in the tree is in the range `[0, 5000]`
- `-1000 <= Node.val <= 1000`
- `-1000 <= targetSum <= 1000`

My intuition: DFS, BFS

In [12]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

from typing import List
class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> List[List[int]]:
        result = []
        def dfs(node: TreeNode, targetSum: int, path: List[int]) -> (None):
            if not node:
                return
            if not node.left and not node.right:
                if targetSum == node.val:
                    result.append(path + [node.val])
                return

            targetSum -= node.val
            dfs(node.left, targetSum, path + [node.val])
            dfs(node.right, targetSum, path + [node.val])
            return
        dfs(root, targetSum, [])
        return result
# Time: O(n)
# Space: O(n*nlogn) larger due to each node create a list

In [13]:
# Calculate Space complexity:
# 1
# 2 2
# 3 3 3 3
# ........

# array elements = 1*2**0 + 2*2**1 + 3*2**2... + d*(2**(d-1))   # d: depth
# d*(2**(d-1)) = ((logn)+1)*2**(logn)                           # d = floor((logn)+1) 
#              = (logn+1)*(n)
#              = nlogn + n
# array elements = 1*2**0 + 2*2**1 + 3*2**2... + d*(2**(d-1))
#                = O(nlogn + nlogn +...+nlogn)
#                = O(n*nlogn)
# Space: O(n*nlogn)

In [14]:
from typing import List
from collections import deque
class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> List[List[int]]:
        def dfs(root: TreeNode, targetSum: int) -> List[List[int]]:
            if not root:
                return []
            if not root.left and not root.right:
                if targetSum == root.val:
                    return [deque([root.val])]
                return []
            targetSum -= root.val
            path = dfs(root.left, targetSum)
            path.extend(dfs(root.right, targetSum))
            for queue in path:
                queue.appendleft(root.val)
            return path
        return [list(queue) for queue in dfs(root, targetSum)]
# Time: O(n)
# Space: O(nlogn) due to d*(2**(d-1)) = nlogn + n

## ✅ Code Quality & Readability

### Strengths:

* Clear problem definition and test cases included.
* Good use of recursion (DFS) to traverse the binary tree.
* Correct functional logic for identifying root-to-leaf paths that match the `targetSum`.
* Excellent use of `deque` in the optimized DFS version to prepend efficiently.
* Concise base cases (`if not root.left and not root.right`) to detect leaf nodes.

---

## ⚙️ Efficiency Review

### Time Complexity:

* **Both versions (list-based and deque-based)** are `O(n)` time where `n = number of nodes`, because each node is visited once.

### Space Complexity:

* **Recursive call stack**: `O(h)`, where `h` is the height of the tree.
* **Path storage**:

  * The original version creates new `path + [node.val]` lists at each recursive call, leading to **`O(n^2)` total auxiliary space** in the worst case (many deep copies).
  * The `deque` version is more memory-efficient with **mutable structures and fewer allocations**, but still ends up with `O(n log n)` due to the number of paths and the average path length in a balanced tree.

---

## 🔍 Detailed Suggestions for Improvement

| Aspect                 | Comments                                                                                    |
| ---------------------- | ------------------------------------------------------------------------------------------- |
| 🔄 `path + [node.val]` | Avoids mutation but causes multiple copies → **optimize with backtracking**.                |
| 🧠 `deque` usage       | Good for prepend efficiency, but converting to list adds overhead.                          |
| 🔁 Function Signature  | Returning `List[deque]` from recursion can be replaced with appending to `result` in-place. |
| 📄 Naming              | `dfs` and `path` are generic; consider `collectPaths` or `currentPath` for clarity.         |
| 📦 Modularity          | Could factor out `is_leaf` and `add_path_to_result` to improve clarity.                     |
| 🧪 Edge Cases          | Nicely tested, including empty trees and no-match cases.                                    |

---

## 🧼 Cleaned-Up & Improved Version (Backtracking):

In [15]:
class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> List[List[int]]:
        result = []

        def dfs(node: TreeNode, remaining: int, path: List[int]) -> None:
            if not node:
                return
            path.append(node.val)

            if not node.left and not node.right and remaining == node.val:
                result.append(path[:])  # Make a copy
            else:
                dfs(node.left, remaining - node.val, path)
                dfs(node.right, remaining - node.val, path)

            path.pop()  # Backtrack

        dfs(root, targetSum, [])
        return result
# Time: O(n)
# Space: O(h)

### ✅ Benefits:

* No new list creation at every recursive level.
* Uses backtracking with `path.pop()` to keep memory usage minimal.
* Very readable; logic follows the flow of traversal naturally.

---

## 🔬 Optional: Iterative Version (if needed for interviews)

In [16]:
from collections import deque

class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> List[List[int]]:
        if not root:
            return []
        
        result = []
        stack = [(root, [root.val], targetSum - root.val)]
        
        while stack:
            node, path, rem = stack.pop()
            if not node.left and not node.right and rem == 0:
                result.append(path)
            if node.right:
                stack.append((node.right, path + [node.right.val], rem - node.right.val))
            if node.left:
                stack.append((node.left, path + [node.left.val], rem - node.left.val))
        
        return result
# Time: O(n)
# Space: O(n*n)

## ✅ Final Notes

| Category        | Rating                                                         |
| --------------- | -------------------------------------------------------------- |
| **Correctness** | ✅ Excellent                                                    |
| **Readability** | ✅ Clear and clean                                              |
| **Efficiency**  | ⚠️ Improved by backtracking                                    |
| **Style**       | ✅ Consistent                                                   |
| **Scalability** | ⚠️ Use mutable list with backtracking to avoid repeated copies |

Would you like to compare space/time trade-offs of recursive vs. iterative versions visually (e.g., chart or table)?

Here is a detailed comparison of the approaches used to solve the "Path Sum II" problem. The table show the time and space complexity trade-offs for different recursive and iterative strategies. If you’re aiming for optimal space usage, the **backtracking recursive approach** is generally preferred.

| Approach                    | Time Complexity | Space Complexity         |
|----------------------------|------------------|---------------------------|
| Recursive (List Copy)          |  O(n)             | O(n²)  |
| Recursive (Backtracking) |  O(n)             | O(h)             |
| Recursive (deque)      |  O(n)             | O(n log n)               |
| Iterative (Stack + Path Copy)     |  O(n)             | O(n²)              |

In [17]:
assert Solution().pathSum(
    TreeNode(5, TreeNode(4, TreeNode(11, TreeNode(7, None, None), TreeNode(2, None, None)), None), TreeNode(8, TreeNode(13, None, None), TreeNode(4, TreeNode(5, None, None), TreeNode(1, None, None)))),
    22
) == [[5,4,11,2],[5,8,4,5]]
assert Solution().pathSum(
    TreeNode(1, TreeNode(2, None, None), TreeNode(3, None, None)),
    5
) == []
assert Solution().pathSum(
    TreeNode(1, TreeNode(2, None, None), None),
    0
) == []
assert Solution().pathSum(
    TreeNode(1, TreeNode(2, None, None), None),
    3
) == [[1,2]]